# node structure 

Ideas: 
1-use the category of the meal as key of the dictionary of the dataset
2- this will be used in coordination with state.meal
3- the dataset will have the following form 
    {
        category : {{object},{object2},{object3}}
    }

# Problems: 
1- we have to check for daily calories so that they're satisfied which translates to checking the calories every three levels which represents a day 

In [11]:
import queue
from copy import deepcopy
import math
from math import sin, cos, radians, atan2, sqrt

# here for the convenience: we chose to add a meal and a day data member.
# using this we can make the check for the daily constraints easier
# for instance for calories we'll just check if the meal is a dinner so that we know that we are at the end of the day and o check if the calories are ok for the day. which we will call a valid state: a valid state is a state that will be considered a worthy state to be expanded. if a state is not valid; we simply won't expand it. ( pruning)




# ------------------------------
# 1. Node Class Definition
# ------------------------------
class Node:
    def __init__(self, state, parent=None, meal=None,day = None,Valid = [], g=0, f=0):
        self.state = state        # Current meal plan state (7x3 matrix)
        self.parent = parent      # Parent node
        self.day = day
        self.meal =meal        # Meal object (if applicable)this will signify what the current category of the state is, will be passed from parent to childre: parent: breakfast -> child: lunch 
        self.depth = 0 if parent is None else parent.depth + 1
        self.g = g                # Cumulative cost from start to this node
        self.f = f                # Evaluation cost (g + heuristic if applicable)
        self.ValidArray = Valid

    def __hash__(self):
        if isinstance(self.state, list):
            state_tuple = tuple([tuple(row) for row in self.state])
            return hash(state_tuple)
        return hash(self.state)

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __gt__(self, other):
        return isinstance(other, Node) and self.f > other.f


# class definition

In [12]:

# ------------------------------
# 2. mealPlanning Class for Food Recommendation System
# ------------------------------

class mealPlanning:
    def __init__(self, initial_state, goal_state,allergies,dietType):
        self.initial_state = initial_state  # Empty meal plan (7x3 matrix)
        self.goal_state = goal_state        # Target meal plan with calorie and cost goals
        self.allergies = allergies  # List of allergies (if any)
        self.diets = dietType
        # Constraints
        self.max_repetitions = 1  # Max times a dish can be repeated in a week
        self.meals_per_day = 3    # Breakfast, Lunch, Dinner
        self.days_per_week = 7    # Monday through Sunday

    def is_goal(self, current_state):
        # cost is here and other global constraints and also the 7 validity checks of nutrients
        pass

    def _is_complete(self, state):
        # Check if all meal slots are filled
        for day in range(self.days_per_week):
            for meal in range(self.meals_per_day):
                if state[day][meal] is None:
                    return False
        return True

    def _calculate_plan_stats(self, state):
        # Calculate total calories and cost for the meal plan
        total_calories = 0
        total_cost = 0
        
        for day in range(self.days_per_week):
            for meal in range(self.meals_per_day):
                dish_id = state[day][meal]
                if dish_id is not None:
                    dish_info = self.state_transition_model[dish_id]
                    total_calories += dish_info['calories']
                    total_cost += dish_info['cost']
                    
        return total_calories, total_cost

    def get_valid_actions(self, current_node): ## we can use sets for optimization, then just do the difference
        #find the category of the meal (i.e breakfast ..Etc)
        meal_idx = current_node.meal
        if meal_idx == "breakfast":
            meal_idx = 0
        elif meal_idx == "lunch":
            meal_idx = 1
        elif meal_idx == "dinner":
            meal_idx = 2
        valid_meals = []
        if meal_idx != 2:
            for meal in Dataset[meal]:
                # Check if the meal appears in the same meal column (breakfast, lunch, or dinner)
                
                ## here i must remind them to put allergy-free when taking the user input in diets
                if not any(current_node.state[day][meal_idx] == meal for day in range(7) if current_node.state[day][meal_idx] is not None) and all(allergy in self.allergies for allergy in meal.allergies) and all(diet in self.diets for diet in meal.diet_type):
                    valid_meals.append(meal)
        else:
            for meal in Dataset[meal]: 
                # Check if the meal appears in the same meal column (breakfast, lunch, or dinner)
                if not any(current_node.state[day][meal_idx] == meal for day in range(7) if current_node.state[day][meal_idx] is not None) and all(allergy in self.allergies for allergy in meal.allergies) and all(diet in self.diets for diet in meal.diet_type):
                    if (self.is_valid_meal(meal, current_node.state[current_node.day])):
                        valid_meals.append(meal)
       
        return valid_meals
    def is_valid_meal(self, meal, DayMeals):
        # Check if the meal is valid based on nutritional balance
        # Replace this with your own nutritional balance check logic
        calories = sum(meal.cal for meal in DayMeals if meal is not None) + meal.cal
        proteins = sum(meal.prot for meal in DayMeals if meal is not None) + meal.prot
        fats = sum(meal.fat for meal in DayMeals if meal is not None) + meal.fat
        carbs = sum(meal.carbs for meal in DayMeals if meal is not None) + meal.carbs
        rating = (sum(meal.rating for meal in DayMeals if meal is not None) + meal.rating ) /3
        daily_calorie_target = self.goal_state.get('cal', 2500)
        daily_protein_target = self.goal_state.get('prot', 75)
        daily_fats_target = self.goal_state.get('fat', 70)
        daily_carbs_target = self.goal_state.get('carb', 150)
        avg_rating = self.goal_state.get('rating', 3)
        

        calorie_margin = daily_calorie_target * 0.1
        protein_margin = daily_protein_target * 0.15
        fat_margin = daily_fats_target * 0.1
        carb_margin = daily_carbs_target * 0.2
        

        calories_valid = abs(calories - daily_calorie_target) <= calorie_margin
        protein_valid = abs(proteins - daily_protein_target) <= protein_margin
        fat_valid = abs(fats - daily_fats_target) <= fat_margin
        carb_valid = abs(carbs - daily_carbs_target) <= carb_margin
        del_valid = rating >= avg_rating

        return calories_valid and protein_valid and fat_valid and carb_valid and del_valid
        
        
        
    def expand_node(self, node, use_cost=False, use_heuristic=False):
        state = node.state
        
        # Find valid actions for this state
        valid_meals = self.get_valid_actions(node)
        ## meal type
        day_idx = node.day
        meal_idx = node.meal
        if meal_idx == "breakfast":
            meal_idx = 0
        elif meal_idx == "lunch":
            meal_idx = 1
        elif meal_idx == "dinner":
            meal_idx = 2
        child_nodes = []
        for meal in valid_meals:
            # Create a new state by placing the dish in the empty slot
            child_state = deepcopy(state)
            child_state[day_idx][meal_idx] = meal
            child = Node(state=child_state, parent=node, action=meal)
            child_nodes.append(child)
            
            
            # for the ones responsible for UCS and A* to figure out
            '''
            # Calculate the new g value (actual cost)
            child_g = node.g + cost
            
            # Calculate the coordinates for heuristic (for meal planning this could be nutritional balance)
            dish_info = self.state_transition_model[dish_id]
            coordinates = [dish_info['calories'], dish_info['nutritional_value']]
            
            # Calculate the total evaluation cost
            total_cost = self.get_total_cost(child_g, coordinates, use_cost, use_heuristic)
            
            # Create the child node
            child_node = Node(
                state=child_state,
                parent=node,
                action=dish_id,
                g=child_g,
                f=total_cost
            )
            
            child_nodes.append(child_node)'''    
        return child_nodes

    def get_total_cost(self, g, coordinates, use_cost, use_heuristic):
        # Calculate the total cost based on actual cost and heuristic
        total = 0
        
        if use_cost:
            total += g  # Add the actual cost
            
        if use_heuristic:
            # Calculate the heuristic (distance to goal)
            calories_remaining = max(0, self.goal_state['calories'] - coordinates[0])
            nutritional_balance = self.goal_state['nutritional_value'] - coordinates[1]
            
            # Weighted combination of calories and nutritional balance
            heuristic = (calories_remaining * self.goal_state['calories_weight'] + 
                         abs(nutritional_balance) * self.goal_state['nutritional_weight'])
            
            total += heuristic
            
        return total

    def get_crow_flies_distance(self, coordinates, goal_coordinates):
        # For food recommendation, this could be nutritional distance
        # More sophisticated implementations might use a multi-dimensional distance metric
        
        # Using Euclidean distance for simplicity
        squared_diff_sum = 0
        for i in range(len(coordinates)):
            squared_diff_sum += (coordinates[i] - goal_coordinates[i]) ** 2
            
        return math.sqrt(squared_diff_sum)

    def print_node(self, node):
        print("Action:", node.action, "| Depth:", node.depth)
        if node.action is not None:
            dish_info = self.state_transition_model[node.action]
            print(f"Added dish: {dish_info['name']}, Calories: {dish_info['calories']}, Cost: {dish_info['cost']}")
        
        # Print the meal plan
        if node.state is not None:
            print("\nCurrent Meal Plan:")
            days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
            meals = ["Breakfast", "Lunch", "Dinner"]
            
            for day_idx, day in enumerate(days):
                print(f"\n{day}:")
                for meal_idx, meal in enumerate(meals):
                    dish_id = node.state[day_idx][meal_idx]
                    if dish_id is not None:
                        dish_info = self.state_transition_model[dish_id]
                        print(f"  {meal}: {dish_info['name']} ({dish_info['calories']} cal, ${dish_info['cost']})")
                    else:
                        print(f"  {meal}: ---")
'''
def is_valid_meal(self, meal, DayMeals):
    # Calculate total nutrients for the day with the new meal
    calories = sum(meal.cal for meal in DayMeals if meal is not None) + meal.cal
    proteins = sum(meal.prot for meal in DayMeals if meal is not None) + meal.prot
    fats = sum(meal.fat for meal in DayMeals if meal is not None) + meal.fat
    carbs = sum(meal.carbs for meal in DayMeals if meal is not None) + meal.carbs
    
    # Define reasonable ranges for daily nutrition
    # These could be user inputs or calculated based on goal state
    daily_calorie_target = self.goal_state.get('daily_calories', 2000)  # Default 2000 calories
    
    # Calculate acceptable ranges (typically 10-20% deviation is acceptable)
    calorie_margin = daily_calorie_target * 0.15  # 15% margin
    
    # Macro distribution targets (percentages of total calories)
    protein_target_pct = self.goal_state.get('protein_pct', 0.25)  # 25% of calories from protein
    fat_target_pct = self.goal_state.get('fat_pct', 0.30)  # 30% of calories from fat
    carb_target_pct = self.goal_state.get('carb_pct', 0.45)  # 45% of calories from carbs
    
    # Convert percentages to grams
    # Protein and carbs: 4 calories per gram, fat: 9 calories per gram
    protein_target_g = (daily_calorie_target * protein_target_pct) / 4
    fat_target_g = (daily_calorie_target * fat_target_pct) / 9
    carb_target_g = (daily_calorie_target * carb_target_pct) / 4
    
    # Define acceptable margins for macronutrients (20% deviation)
    protein_margin = protein_target_g * 0.20
    fat_margin = fat_target_g * 0.20
    carb_margin = carb_target_g * 0.20
    
    # Check if values are within acceptable ranges
    calories_valid = abs(calories - daily_calorie_target) <= calorie_margin
    protein_valid = abs(proteins - protein_target_g) <= protein_margin
    fat_valid = abs(fats - fat_target_g) <= fat_margin
    carb_valid = abs(carbs - carb_target_g) <= carb_margin
    
    # If we're checking dinner (the last meal of the day), be strict about totals
    # For breakfast and lunch, we should be more lenient since other meals will follow
    if len([m for m in DayMeals if m is not None]) == 2:  # This is dinner
        return calories_valid and protein_valid and fat_valid and carb_valid
    else:  # This is breakfast or lunch
        # For earlier meals, just make sure we're not exceeding daily targets
        return calories <= daily_calorie_target * 1.1 and proteins <= protein_target_g * 1.2

def init(self, initial_state, goal_state, allergies, dietType, 
             calorie_margin_pct=0.15, protein_margin_pct=0.20, fat_margin_pct=0.20, carb_margin_pct=0.20):
    # Existing initialization code...
    self.calorie_margin_pct = calorie_margin_pct
    self.protein_margin_pct = protein_margin_pct 
    self.fat_margin_pct = fat_margin_pct
    self.carb_margin_pct = carb_margin_pct


def expand_node(self, node, use_cost=True, use_heuristic=False):
    # If the node represents a complete plan, don't expand further
    if node.day >= self.days_per_week:
        return []
    
    # Determine which meal type we're filling next
    meal_types = ["breakfast", "lunch", "dinner"]
    meal_type = meal_types[node.meal]
    
    # Get valid meals for this position
    valid_meals = self.get_valid_actions(node)
    
    # Generate child nodes
    children = []
    for meal in valid_meals:
        # Create a new state by copying the current state
        new_state = [row[:] for row in node.state]
        
        # Add the meal to the state
        new_state[node.day][node.meal] = meal
        
        # Determine the next position (day and meal)
        next_meal = (node.meal + 1) % 3
        next_day = node.day + 1 if next_meal == 0 else node.day
        
        # Calculate cost (could be financial cost, nutritional cost, etc.)
        # For now, using meal cost as the cost
        cost = node.cost + meal.cost
        
        # Calculate heuristic value if needed
        heuristic = 0
        if use_heuristic:
            # Remaining days * average daily cost as a simple heuristic
            remaining_days = self.days_per_week - next_day
            avg_daily_cost = self.goal_state.get('budget', 0) / self.days_per_week  
            heuristic = remaining_days * avg_daily_cost
            
            # Adjust heuristic based on how well we're meeting daily nutritional targets
            if node.meal == 2:  # After adding dinner, evaluate day completeness
                day_nutrients = self._calculate_day_nutrients(new_state[node.day])
                nutrient_balance = self._evaluate_nutrient_balance(day_nutrients)
                # Penalize days with poor nutritional balance
                heuristic += (1 - nutrient_balance) * 50  # Higher penalty for poor balance
        
        # Calculate f value (g + h)
        f_value = cost + heuristic if use_heuristic else cost
        
        # Create child node
        child = Node(
            state=new_state,
            day=next_day,
            meal=next_meal,
            cost=cost,
            f_value=f_value,
            parent=node,
            action=meal
        )
        
        children.append(child)
    
    return children



'''

# Example usage with a toy meal planning problem
def toy_test_meal_planning_problem():
    # Define a database of dishes with their properties
    dishes_database = {
        'D1': {
            'name': 'Oatmeal with Berries',
            'cal': 300,            
            'prot': 300,            
            'carbs': 300,            
            'fat': 300,            
            'rating': 300,
            'cost': 3.50,
            'nutritional_value': 8,
            'category': 'breakfast'
        },
        'D2': {
            'name': 'Scrambled Eggs & Toast',
            'cal': 400,            
            'prot': 400,            
            'carbs': 400,            
            'fat': 400,            
            'rating': 400,
            'cost': 4.00,
            'nutritional_value': 7,
            'category': 'breakfast'
        },
        'D3': {
            'name': 'Greek Salad',
            'cal': 350,            
            'prot': 350,            
            'carbs': 350,            
            'fat': 350,            
            'rating': 350,
            'cost': 6.00,
            'nutritional_value': 9,
            'category': 'lunch'
        },
        'D4': {
            'name': 'Chicken Sandwich',
            'cal': 550,            
            'prot': 550,            
            'carbs': 550,            
            'fat': 550,            
            'rating': 550,
            'cost': 7.50,
            'nutritional_value': 6,
            'category': 'lunch'
        },
        'D5': {
            'name': 'Grilled Salmon',
            'cal': 450,            
            'prot': 450,            
            'carbs': 450,            
            'fat': 450,            
            'rating': 450,
            'cost': 9.00,
            'nutritional_value': 10,
            'category': 'dinner'
        },
        'D6': {
            'name': 'Vegetable Stir-Fry',
            'cal': 380,            
            'prot': 380,            
            'carbs': 380,            
            'fat': 380,            
            'rating': 380,
            'cost': 5.50,
            'nutritional_value': 8,
            'category': 'dinner'
        }
    }
    
    # Define the initial state: empty 7x3 matrix (7 days, 3 meals per day)
    initial_state = [[None for _ in range(3)] for _ in range(7)]
    # to be taken from the user input
    
    # Define the goal state: target calories and cost for the week
    goal_state = {
        'calories': 8400,  # Average 1200 calories per day
        'calories_margin': 0.1,  # Acceptable margin of error
        'proteins_margin': 0.15,  # Acceptable margin of error
        'carbs_margin': 0.2,  # Acceptable margin of error
        'fats_margin': 0.1,  # Acceptable margin of error
        'cost': 140.00,  # Weekly food budget
        'cost_margin': 0.15,  # Acceptable margin of error
        'avg_rating': 3
        }
    
    # Create the problem instance
    meal_plan_problem = mealPlanning(
        initial_state=initial_state,
        goal_state=goal_state,
        allergies=[],
        dietType=[]
    )
    
    # Test the problem methods
    print("Testing meal planning problem:")
    
    # Test is_goal method
    print("\nTesting is_goal:")
    print("Is empty plan a goal?", meal_plan_problem.is_goal(initial_state))  # Should be False
    
    
    # Test expand_node
    print("\nTesting expand_node:")
    start_node = Node(state=initial_state, g=0)
    children = meal_plan_problem.expand_node(start_node)
    print(f"Number of child nodes: {len(children)}")
    
    # Test print_node
    print("\nTesting print_node:")
    meal_plan_problem.print_node(start_node)
    
    if children:
        print("\nSample child node:")
        meal_plan_problem.print_node(children[0])
    

if __name__ == "__main__":
    # Run the toy test for meal planning
    toy_test_meal_planning_problem()

Testing meal planning problem:

Testing is_goal:
Is empty plan a goal? None

Testing expand_node:


NameError: name 'Dataset' is not defined